In [ ]:
import pandas.io.sql as pd_sql
import psycopg2 as pg
import pprint as pp
import pandas as pd

In [ ]:
df = pd.read_excel('default of credit card clients.xls', index_col=0, header=1)  

In [ ]:
df.rename(columns ={'LIMIT_BAL': 'CREDIT_LINE',
                    'PAY_0': 'STATUS_2005_04',
                    'PAY_2': 'STATUS_2005_05',
                    'PAY_3': 'STATUS_2005_06',
                    'PAY_4': 'STATUS_2005_07',
                    'PAY_5': 'STATUS_2005_08',
                    'PAY_6': 'STATUS_2005_09', 
                    'BILL_AMT1': 'BILL_2005_09',
                    'BILL_AMT2': 'BILL_2005_08',
                    'BILL_AMT3': 'BILL_2005_07',
                    'BILL_AMT4': 'BILL_2005_06',
                    'BILL_AMT5': 'BILL_2005_05',
                    'BILL_AMT6': 'BILL_2005_04', 
                    'PAY_AMT1': "PAYMENT_2005_09", 
                    'PAY_AMT2': "PAYMENT_2005_08", 
                    'PAY_AMT3': "PAYMENT_2005_07", 
                    'PAY_AMT4': "PAYMENT_2005_06", 
                    'PAY_AMT5': "PAYMENT_2005_05", 
                    'PAY_AMT6': "PAYMENT_2005_04"}, inplace = True)

In [ ]:
def create_table_sql(df, table_name):
    column_names = df.dtypes.index.values
    column_types = df.dtypes.values
    
    table_name = table_name.lower()
    
    create_table = "CREATE TABLE IF NOT EXISTS %s (\n" %table_name
    
    for index, column_name in enumerate(column_names):
        column_type = column_types[index]
        column_name = column_name.lower().replace(' ', '_')
        create_table += '\t'
        
        if column_type == 'int64':
            create_table = create_table + column_name + " INT DEFAULT NULL, \n"
        elif column_type == 'object':
            create_table = create_table + column_name + " VARCHAR DEFAULT NULL, \n"
        elif column_type == 'float64':
            create_table = create_table + column_name + " DECIMAL DEFAULT NULL, \n"
        
        if index == len(column_names) - 1:
            create_table = create_table[:-3] 
            
    create_table += "\n);"
        
    print(create_table)
    return create_table

In [ ]:
create_table = create_table_sql(df, "ccdefault")

In [ ]:
connection_args = {'host': '204.236.167.183',
                   'user': 'ubuntu',
                   'port': 5432
                  }

connection = pg.connect(**connection_args)
cursor = connection.cursor()

In [ ]:
db_name = 'ccdefault'
cursor.execute('CREATE DATABASE %s ;' %db_name)
cursor.execute('commit;')
connection.close()

In [ ]:
#cursor.execute('rollback;')

In [ ]:
## Connecting to database "ccdefault"
connection_args['dbname'] = 'ccdefault'
connection = pg.connect(**connection_args)
cursor = connection.cursor()

In [ ]:
#cursor.execute("DROP TABLE ccdefault")

In [ ]:
cursor.execute(create_table)

In [ ]:
#get the file path of the data
csv_file_path = '/home/ubuntu/Project_3/ccdefault.csv'

#define the table name
table_name = 'ccdefault'

#COPY statement in psql as string
copy_statement = "COPY " + table_name + " FROM '" + csv_file_path + "' DELIMITER ',' CSV HEADER;"

print(copy_statement)

In [ ]:
#copy data to psql table
cursor.execute(copy_statement)

#save data to psql table
connection.commit()

In [ ]:
query = "SELECT * \
         FROM ccdefault \
         ;"

#run the query
cursor.execute(query)

#fetch the results
results = list(cursor.fetchall())
pprint(results)

In [ ]:
df = pd_sql.read_sql_query(query, connection)

In [ ]:
cursor.close()
connection.close()